# 8. Feature selection

In [1]:
import scanpy as sc
import anndata2ri
import logging
import numpy as np

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [3]:
%%R
BiocManager::install("scry", update = FALSE)
library(scry)


Pobrane pakiety binarne są w
	/var/folders/x1/7112ysln0nl2vyr99j1nllf40000gp/T//Rtmp6cQkdg/downloaded_packages


In [4]:
adata = sc.read(
    '../../0_outs/sample-SRR15058452/SRR15058452_normalization.h5ad'
)

In [5]:
ro.globalenv["adata"] = adata

In [6]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

/Users/adrian/opt/anaconda3/envs/mamba/envs/simpleaf/lib/python3.10/site-packages/anndata2ri/r2py.py:106: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm or None, layers=layers)


In [7]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T

In [8]:
idx = binomial_deviance.argsort()[-4000:]
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[idx] = True

adata.var["highly_deviant"] = mask
adata.var["binomial_deviance"] = binomial_deviance

In [9]:
adata.write("../../0_outs/sample-SRR15058452/SRR15058452_feature_selection.h5ad")